## Example of using Pure to check data quality

This example shows how Pure framework can be used to check the quality of data for some datasets. 

In [2]:
import sys
sys.path.append('..')

In [3]:
import pandas as pd
import numpy as np

from pure.report import Report

#### For the first example let's take dataset from kaggle competitions with House Prices Dataset and Context Ad Clicks Dataset: 

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

https://www.kaggle.com/datasets/arashnic/ctrtest

In [17]:
house_prices_data = pd.read_csv('./example_data/house_prices.csv')
clicks_data = pd.read_csv('./example_data/clicks.csv')

In [18]:
house_prices_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [19]:
clicks_data.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b92382...,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbd...,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b08607...,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a375633...,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072...,2018-11-15 00:02:00,62873,473,latest,0,0


### Report

The main application of the Pure project is to provide an opportunity to run some lists of metrics and give information about what metrics are passed, failed or dropped with errors.

For this purpose there is a Report class that takes a dictionary of datasets and apply metrics from the checklist (explained later) to them.

At first, create tables that we want to apply metrics to.

In [20]:
tables = {"house_prices": house_prices_data, "clicks": clicks_data}

Import metrics that we want to check for our datasets

In [34]:
from pure.metrics import CountTotal, CountZeros, CountNull, CountLag

Create checklist for the report class with metrics we want to check and what restrictions (limits) we want to control for resulting values of these metrics.

Checklist is a list of tuples containing:

'name of the table': str,

'metric class': Metric,

'limits dict': Dict[str, Tuple[float, float]] (defines limits for the field of the metric result we want to control).

It's needed for initialization of the Report class.

In [35]:
checklist = [
    ("house_prices", CountTotal(), {"total": (1, 1e6)}),
    ("house_prices", CountZeros("MiscVal"), {"delta": (0, 0.3)}),
    ("house_prices", CountNull(["LotFrontage"]), {"delta": (0, 0.2)}),
    ("clicks", CountTotal(), {"total": (1, 1e6)}),
    ("clicks", CountZeros('is_4G'), {"delta": (0, 0.1)}),
    ("clicks", CountLag('impression_time'), {})
]

Report class has two methods we should be familiar with:

``fit`` -- applies metrics to the datasets and check if the resulting values match the limits defined in checklist; \
``to_str`` -- transform result of the fit method to a more readble form.

Let's create an instance with setting engine we want ('pandas' or 'pyspark'). For us it's 'pandas'

In [36]:
report = Report(checklist=checklist, engine='pandas')

Fit report

In [37]:
result = report.fit(tables)

Let's print resulting report info

In [38]:
result_to_str = report.to_str()
print(result_to_str)

DQ Report for tables ['clicks', 'house_prices']

     table_name               metric               limits               values status error
0  house_prices         CountTotal()  {'total': (1, 10...      {'total': 1460}      .      
1  house_prices  CountZeros(colum...  {'delta': (0, 0.3)}  {'total': 1460, ...      F      
2  house_prices  CountNull(column...  {'delta': (0, 0.2)}  {'total': 1460, ...      .      
3        clicks         CountTotal()  {'total': (1, 10...    {'total': 237609}      .      
4        clicks  CountZeros(colum...  {'delta': (0, 0.1)}  {'total': 237609...      F      
5        clicks  CountLag(column=...                   {}  {'today': '2023-...      .      

Passed: 4 (66.67%)
Failed: 2 (33.33%)
Errors: 0 (0.0%)

Total: 6


#### Explanation of the report result

In the report result we see two metrics passed with 'F' status -- it means that value 'delta' in the result of metrics doesn't match limits defined for it. All others are passed without any errors. 

'status' column:

‘.’ - if the check is successful; 

'F' - if the check is not successful; 

'E' - if an error occurred during execution

### Metrics

In case we want just apply some metric to our dataset and see the result value.

Import metric we want to check.

For example, let it be CountCB metric that calculates confidence bounds for chosen column and CountLag that calculates a lag between last date and today.

In [39]:
from pure.metrics import CountCB, CountLag

Create an instances of metrics

In [40]:
count_cb_metric = CountCB(column="LotArea", conf=0.95)
count_lag_metric = CountLag(column="impression_time")

Calculate the results

In [42]:
bounds = count_cb_metric(house_prices_data)
print(bounds)

{'lcb': 2298.0250000000005, 'ucb': 22698.249999999927}


In [43]:
lag_result = count_lag_metric(clicks_data)
print(lag_result)

{'today': '2023-05-15', 'last_day': '2018-12-13', 'lag': 1614}
